<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li></ul></li><li><span><a href="#Experiment-with-Terrain-Routing" data-toc-modified-id="Experiment-with-Terrain-Routing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Experiment with Terrain Routing</a></span><ul class="toc-item"><li><span><a href="#Background" data-toc-modified-id="Background-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Create-experiment-simulation-directory" data-toc-modified-id="Create-experiment-simulation-directory-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create experiment simulation directory</a></span></li><li><span><a href="#Edit-the-namelists-and-run-the-simulation" data-toc-modified-id="Edit-the-namelists-and-run-the-simulation-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Edit the namelists and run the simulation</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Discussion</a></span></li></ul></li></ul></div>

# Lesson 5 - Land Surface Experiments
## Overview
In Lessons 4 we experimented with different initial conditions and precipitation data. We also used NCO commands to modify parameters and compare against defaults. In this lesson, we will bring a few of these components together into a combined parameter change and physics experiment.

**NOTE: If you have not completed Lessons 1 through 4, please stop and do so now.**

In particular, this lesson builds from steps completed in Lesson 4.

### Software and conventions
The easiest way to run these lessons is via the [wrfhydro/nwm-training](https://cloud.docker.com/u/wrfhydro/repository/docker/wrfhydro/nwm-training) Docker container, which has all software dependencies and data pre-installed.

For a complete description of the software environment used for this training please see [Lesson 0 - Getting started](Lesson-0-start.ipynb).

You may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in Jupyter Lab by selecting `New -> Terminal` in your `Home` tab of Jupyter Lab and input the commands manually if you prefer. You can also use your own terminal by logging into the container with the command `docker exec -it wrf-hydro-training bash`

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session. 

## Experiment with Terrain Routing 
### Background
In most global and regional land surface models, model cells are simulated independently and "runoff" is interpreted as excess surface and subsurface water that is "scraped" from each cell after vertical infiltration and drainage processes complete. WRF-Hydro adds functionality to allow this excess water to instead move laterally around the land surface based on topographic and head gradients. WRF-Hydro can explicitly represent overland flow processes, where infiltration and saturation excess water propagates over the surface through a diffusive wave formulation. In addition, WRF-Hydro includes shallow subsurface flow, allowing saturated soil water move from cell to cell through a Boussinesq approximation. For more detailed information refer to [WRF-Hydro V5 Technical Description](https://ral.ucar.edu/sites/default/files/public/WRF-HydroV5TechnicalDescription.pdf). 

### Objective
Adjust two key parameters impacting water partitioning and lateral flow, then compare hydrographs with and without terrain routing processes active.

### Create experiment simulation directory
This lesson will use the compiled binary (`wrf_hydro.exe`) and template directory (`lesson4/run_NWM_baseline`) from Lesson 4. First, create a copy of the Lesson 4 template directory to use for a parameter manipulation experiment.

In [ ]:
%%bash
mkdir -p ~/wrf-hydro-training/output/lesson5/
cp -r ~/wrf-hydro-training/output/lesson4/run_NWM_template \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on

### Modify surface infiltration and lateral conductivity parameters

#### REFKDT
There are a number of model parameters that affect lateral terrain flow. One important Noah/NoahMP parameter that we commonly adjust when activating terrain routing is *refkdt*. *refkdt* controls how easily precipitation reaching the surface infiltrates into the soil column vs. staying on the surface where it will be "scraped" off as surface runoff. Higher values of *refkdt* lead to more infiltration and less surface runoff. This tunable parameter is set by default to a relatively high value suitable for running the column land surface model only. When we activate terrain routing and explicitly model these processes, we often reduce this parameter. In addition, if you are calling the land surface model on a small timestep (e.g., seconds to minutes), you may want to reduce this parameter to compensate for the more frequent calls to the vertical infiltration scheme. 

**Step 1: Use the NCO command `ncap2` to modify the *refkdt* parameter values in the `soil_properties.nc` file**

NCO (NetCDF Operators, http://nco.sourceforge.net/) is a set of useful utilities to manipulate NetCDF files. We will use the NCO command `ncap2` to reduce the *refkdt* value to 2.0.

First, we check the current parameter values using `ncks`.

In [ ]:
%%bash
ncdump -v refkdt ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/soil_properties.nc \
| tail -n 10

Then, we modify the parameter values using `ncap2` and confirm our changes.

In [ ]:
%%bash
ncap2 -O -s "refkdt=refkdt*0.0+1.5" \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/soil_properties.nc
ncdump -v refkdt ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/soil_properties.nc \
| tail -n 10

#### LKSATFAC
The Fulldom_hires.nc file contains two parameters that are also important for lateral flow processes. The *LKSATFAC* parameter is a multiplier on the prescribed lateral hydraulic conductivity values specified in `hydro2dtbl.nc` (or in older model versions, `HYDRO.TBL`). By default, lateral conductivity in `hydro2dtbl.nc` or `HYDRO.TBL` matches vertical conductivity specified in `soil_properties.nc` or `SOILPARM.TBL`. However, in the real world we frequently see many orders of magnitude higher conductivities in the lateral direction vs. the vertical direction (due to soil stratigraphy, preferential flowpaths caused by roots and animals, etc.). *LKSATFAC* is an easy way to adjust this anisotropy, and by default it is set to 1,000.

**Step 2: Use the NCO command `ncap2` to modify the *LKSATFAC* parameter values in the `Fulldom_hires.nc` file**

We will use the NCO command `ncap2` to reduce the *LKSATFAC* value to 100.0.

First, we check the current parameter values.

In [ ]:
%%bash
ncdump -v LKSATFAC ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/Fulldom_hires.nc \
| tail -n 10

Then, we modify the parameter values using `ncap2` and confirm our changes.

In [ ]:
%%bash
ncap2 -O -s "LKSATFAC=LKSATFAC*0.0+500.0" \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/Fulldom_hires.nc \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/Fulldom_hires.nc
ncdump -v LKSATFAC ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/DOMAIN/Fulldom_hires.nc \
| tail -n 10

### Edit the namelists and run the "terrain routing on" simulation
To focus on the terrain routing processes, we will deactivate the deeper groundwater bucket model and just use the pass-through option, as demonstrated in Experiment 3 in Lesson 4.

**Step 1: Edit the `hydro.namelist` file in wrf-hydro-training/output/lesson6/run_terrain_params_routing_on **

Change the option `GWBASESWCRT = 1` for exponential bucket model to `GWBASESWCRT = 2` for the pass-through simulation

```
! Switch to activate baseflow bucket model...(0=none, 1=exp. bucket, 2=pass-through)
GWBASESWCRT = 2
```

**Step 2: Create a copy of the directory to use for our second experiment**
Now that our simulation directory is setup, we will make a quick copy to use for our "terrain routing off" experiment BEFORE we run the simulation (so we don't copy all of the extra output files).

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on \
~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off

**Step 3: Run the simulation**

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

**Step 4: Check that the simulation finished successfully**

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/diag_hydro.00000

### Edit the namelists and run the "terrain routing off" simulation
Now, we run the same experiment as above but with the terrain routing modules turned off. We first check the `run_terrain_params_routing_off` directory that we created above to make sure we have all of the needed model simulation components.

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off

We will make two modifications to `hydro.namelist` to turn off the terrain routing modules. 

**Step 1: Edit the `hydro.namelist` file**

For this experiment, we will set the *OVRTSWCRT* and *SUBRTSWCRT* physics options to 0, which deactivates both overland and subsurface terrain routing modules. 

```
! Switch to activate subsurface routing...(0=no, 1=yes)
SUBRTSWCRT = 0
```

```
! Switch to activate surface overland flow routing...(0=no, 1=yes)
OVRTSWCRT = 0
```

**Step 2: Run the simulation**

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

**Step 3: Check that the simulation finished successfully**

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with terrain routing on and our experiment with terrain routing off.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see Lesson 3.

**Load the xarray python package**

In [ ]:
# Load the xarray package
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

**Load the CHANOBS streamflow datasets**

We are going to use the CHANOBS files because it will limit the number of grid cells to only those which we have specified have a gage.

In [ ]:
chanobs_control = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson4/run_gw_passthrough/*CHANOBS*',
                            concat_dim='time')
chanobs_terrain_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/*CHANOBS*',
                            concat_dim='time')
chanobs_terrain_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off/*CHANOBS*',
                            concat_dim='time')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the hydrographs**

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for subsurface routing on and off',fontsize=24)
chanobs_control.sel(feature_id = 2).streamflow.plot(label='Control',
                                                        color='black',
                                                        linestyle='--')
chanobs_terrain_on.sel(feature_id = 2).streamflow.plot(label='Parameter Change, Terrain Routing On',
                                                        color='blue',
                                                        linestyle='-')
chanobs_terrain_off.sel(feature_id = 2).streamflow.plot(label='Parameter Change, Terrain Routing Off',
                                                        color='red',
                                                        linestyle='-')
obs[obs['site_no'] == '01374581'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,85)
plt.legend()
plt.show()

**Load the LDASOUT land model output files**

In [ ]:
ldasout_terrain_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_terrain_params_routing_on/*LDASOUT*',
                            concat_dim='time')
ldasout_terrain_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson6/run_terrain_params_routing_off/*LDASOUT*',
                            concat_dim='time')

**Plot the soil moisture states**

In [ ]:
# Select data for 2011-08-28 16Z at surface layer
ldasout_terrain_on_time = ldasout_terrain_on.sel(time = '2011-08-28T16:00:00').sel(soil_layers_stag = 0)
ldasout_terrain_off_time = ldasout_terrain_off.sel(time = '2011-08-28T16:00:00').sel(soil_layers_stag = 0)

# We will narrow the range of plotted soil moisture values to better see some of the gradients
fig, axes = plt.subplots(ncols=2,figsize=(12, 6))
plt.suptitle('Surface soil moisture states for terrain routing on and off',fontsize=24)
ldasout_terrain_on_time.SOIL_M.plot(ax=axes[0],vmin=0.30,vmax=0.45,cmap='viridis_r')
axes[0].set_title('Terrain routing on')
ldasout_terrain_off_time.SOIL_M.plot(ax=axes[1],vmin=0.30,vmax=0.45,cmap='viridis_r')
axes[1].set_title('Terrain routing off')
plt.show()

### Discussion
When terrain routing options are turned off, we often see higher, flashier streamflow responses. With lateral redistribution of flows activated, surface water has an opportunity to infiltrate into neighboring cells with lower water tables. Therefore more water generally goes to soil storage (and potentially evapotranspiration) and less to runoff. You also see the impact of the *refkdt* parameter, which has potential to somewhat offset this re-infiltration effect, though we may not see the same spatial patterns in resulting soil moisture states.

# Next up - Do it yourself!
This concludes Lesson 6. Spend some time creating your own parameter and physics experiments.

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `KERNAL->SHUTDOWN` in the jupyter notebook toolbar.**